In [2]:
import deep_image_matching as dim
import yaml

logger = dim.setup_logger("dim")

params = {
    "dir": "./datasets/belv_20230725",
    "pipeline": "superpoint+lightglue",
    "config_file": "./datasets/belv_20230725/superpoint+lightglue.yaml",
    "strategy": "bruteforce",
    "quality": "high",
    "tiling": "none",
    "skip_reconstruction": False,
    "force": True,
    "camera_options": "./datasets/belv_20230725/cameras.yaml",
    "openmvg": None,
    "verbose": True,
}

# Build configuration
config = dim.Config(params)
config.save()

Using a custom configuration file: /home/francesco/phd/deep-image-matching/datasets/belv_20230725/superpoint+lightglue.yaml
2024-04-09 18:33:58 | [WARNING ] Extractor name in configuration file /home/francesco/phd/deep-image-matching/datasets/belv_20230725/superpoint+lightglue.yaml does not match with the extractor chosen from CLI or GUI. The custom configuration is not set, but matching is run with the default options.
2024-04-09 18:33:58 | [WARNING ] Matcher name in configuration file /home/francesco/phd/deep-image-matching/datasets/belv_20230725/superpoint+lightglue.yaml does not match with the matcher chosen from CLI or GUI. The custom configuration is not set, but matching is run with the default options.
Config general:
{'camera_options': './datasets/belv_20230725/cameras.yaml',
 'db_path': None,
 'geom_verification': <GeometricVerification.PYDEGENSAC: 1>,
 'graph': True,
 'gv_confidence': 0.99999,
 'gv_threshold': 4,
 'image_dir': PosixPath('datasets/belv_20230725/images'),
 'ma

In [3]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

# Read camera options
with open(config.general["camera_options"], "r") as file:
    camera_options = yaml.safe_load(file)

# Export in colmap format
database_path = config.general["output_dir"] / "database.db"
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)

2024-04-09 18:34:04 | [DEBUG   ] Matching options: Quality: HIGH - Tiling: NONE
2024-04-09 18:34:04 | [DEBUG   ] Saving directory: datasets/belv_20230725/results_sift+kornia_matcher_bruteforce_quality_high
2024-04-09 18:34:04 | [DEBUG   ] Running inference on device cuda
Loaded SuperPoint model
2024-04-09 18:34:05 | [DEBUG   ] Matching options: Tiling: NONE
2024-04-09 18:34:05 | [DEBUG   ] Saving directory: datasets/belv_20230725/results_sift+kornia_matcher_bruteforce_quality_high
2024-04-09 18:34:05 | [DEBUG   ] Running inference on device cuda
2024-04-09 18:34:06 | [INFO    ] Running image matching with the following configuration:
2024-04-09 18:34:06 | [INFO    ]   Image folder: datasets/belv_20230725/images
2024-04-09 18:34:06 | [INFO    ]   Output folder: datasets/belv_20230725/results_sift+kornia_matcher_bruteforce_quality_high
2024-04-09 18:34:06 | [INFO    ]   Number of images: 2
2024-04-09 18:34:06 | [INFO    ]   Matching strategy: bruteforce
2024-04-09 18:34:06 | [INFO    ]  

  0%|          | 0/2 [00:03<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.72 GiB. GPU 0 has a total capacity of 11.75 GiB of which 809.00 MiB is free. Process 276581 has 141.49 MiB memory in use. Process 298292 has 2.66 GiB memory in use. Including non-PyTorch memory, this process has 7.20 GiB memory in use. Of the allocated memory 5.86 GiB is allocated by PyTorch, and 17.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
# Export in colmap format
database_path = output_dir / "database.db"
dim.io.export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)

NameError: name 'output_dir' is not defined

In [69]:
# Set camera options

import numpy as np

cameras = {
    1: {
        "model": 4,  # 4 is OPENCV model
        "width": 6000,
        "height": 4000,
        "params": [
            9.26789262766209504e03,
            9.26789262766209504e03,
            3.05349107994520591e03,
            1.94835654532114540e03,
            -8.07042713029020586e-02,
            9.46617629940955385e-02,
            3.31782983128223608e-04,
            -4.32106111976037410e-04,
        ],
    },
    2: {
        "model": 4,
        "width": 6000,
        "height": 4000,
        "params": [
            6.62174345720628298e03,
            6.62174345720628298e03,
            3.01324420057086490e03,
            1.94347461466223308e03,
            -9.41830394356213407e-02,
            8.55303528514532035e-02,
            1.68948638308769863e-04,
            -8.74637609310216697e-04,
        ],
    },
}
with dim.utils.COLMAPDatabase(database_path) as db:
    for id, cam in cameras.items():
        db.update_camera(
            camera_id=id,
            model=cam["model"],
            width=cam["width"],
            height=cam["height"],
            params=cam["params"],
        )

In [70]:
# import pycolmap

# print(pycolmap.IncrementalPipelineOptions().summary())

In [64]:
# Run reconstruction
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=0.5,
    ),
    mapper=dict(filter_min_tri_angle=0.5, filter_max_reproj_error=5.0),
)
refine_intrinsics = False
verbose = False

model = dim.reconstruction.pycolmap_reconstruction(
    database_path=output_dir / "database.db",
    sfm_dir=output_dir,
    image_dir=imgs_dir,
    refine_intrinsics=refine_intrinsics,
    options=opt,
    verbose=verbose,
)

2024-04-09 15:15:21 | [INFO    ] Running 3D reconstruction...


I20240409 15:15:21.422063 167579 misc.cc:198] 
Loading database
I20240409 15:15:21.422781 167579 database_cache.cc:54] Loading cameras...
I20240409 15:15:21.422809 167579 database_cache.cc:64]  2 in 0.000s
I20240409 15:15:21.422822 167579 database_cache.cc:72] Loading matches...
I20240409 15:15:21.422835 167579 database_cache.cc:78]  1 in 0.000s
I20240409 15:15:21.422838 167579 database_cache.cc:94] Loading images...
I20240409 15:15:21.423250 167579 database_cache.cc:143]  2 in 0.000s (connected 2)
I20240409 15:15:21.423261 167579 database_cache.cc:154] Building correspondence graph...
I20240409 15:15:21.423429 167579 database_cache.cc:190]  in 0.000s (ignored 0)
I20240409 15:15:21.423439 167579 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240409 15:15:21.423715 167579 misc.cc:198] 
Finding good initial image pair
I20240409 15:15:21.490808 167579 misc.cc:198] 
Initializing with image pair #1 and #2
I20240409 15:15:21.491742 167579 misc.cc:198] 
Global bundle adjustment
I20240409 15:15

2024-04-09 15:15:21 | [INFO    ] Reconstructed 1 model(s).
2024-04-09 15:15:21 | [INFO    ] Largest model is #0 with 2 images.
2024-04-09 15:15:21 | [INFO    ] Exporting model...


I20240409 15:15:21.692776 167579 timer.cc:91] Elapsed time: 0.005 [minutes]
